# Instruções

## Primeiros Passos

A não ser que você tenha conhecimento prévio de programação, recomendo que sejam executadas todas as células antes que comecem a utilizar o caderno. Isto fará com que não haja nenhuma dependência não instalada ou função não declarada.  
Para isso recomendo que clique em Kernel, entre *Cell* e *Widjets*, na barra abaixo do logo do Jupyter e, em seguida, *Restart & Run All*.  

## Entendendo os Exemplos

Além dos exemplos padrão, que utilizarei para demonstrar o funcionamento do programa, também deixarei comentados templates de chamada de função, que vocês utilizarão para criar suas próprias macros!

Um exemplo consiste em uma chamada de função com os valores já postos que retorna um resultado provavelmente muito próximo dos seus próprios resultados. Para executar um exemplo vá até a célula onde ele está e aperte CTRL+Enter, o resultado aparecerá abaixo da célula

## Entendendo os Templates

Os templates estão aqui para guiar vocês a montarem suas próprias chamas de funções para criar macros pras suas fichas. Templates estão organizados da seguinte forma:

```cria_descricao("{tipo_acao}", "{id_acao}", "{copia}")```  
```cria_macro("{tipo_acao}", "{id_acao}", "{copia}")```  
-> Para ações de Ataque, Habilidade e Poder  

```cria_descricao("{tipo_acao}", "{id_acao}", "{copia}", circulo="{circulo_magia}")```   
```cria_macro("{tipo_acao}", "{id_acao}", "{copia}", circulo="{circulo_magia}")```  
-> Para ações de Magia

Os tipo de ação aceita são:
    
    * ability
    * attack
    * power
    * spell
    
Cópia indica ao programa se você deseja ou não que o resultado da função seja copiado para seu CTRL+C, isso tem o único objetivo de facilitar na criação das macros. As entradas para copia aceitas são:
    
    * sim
    * nao

Os circulos de magia vão de 1 até 5.
 
O ID da ação é o ID armazenado pelo Roll 20. Para acessá-lo, clique com o botão direito na linha que contem a acão para qual você deseja criar a macro e procurar a div com classe ```rep_item```. Em seguida copie o valor sob o atributo ```data-reprowid```, este é seu ID.

Para o tipo de template *create_macro*, ao criar uma ação do tipo Ataque, você pode informar um parâmetro adicional ```critico```que indica como o dano do acerto crítico será calculado. Os valores aceitos para crítico são:

    * DANO
        > Multiplicando o valor dos dados pelo multiplicador de crítico
    * DADO
        > Multiplicando o número de dados do ataque


Recomendo, como antes, caso ainda esteja confuso sobre como acessar o ID da ação, que assista o vídeo https://youtu.be/dGK5eYLgz3E?t=549, do Vinícius silva para mais informações.
PS: Isso é bem mais simples do que parece, e já posicionei o vídeo no momento que ele fala como pegar esse ID, que é a única coisa necessária para nossos objetivos.
    
## Criando uma Macro

Para criar uma macro, realize as substituições indicadas no tópico anterior e role sua célula com a chamada de função apertando CTRL+Enter enquanto estiver com ela selecionada.


## Finalizações

Com o tempo vocês se tornarão mais familiarizados com os notebooks pra perceber que podem realizar pequenas alterações nos templates fornecidos são apenas sujestões e que vocês podem criar seus próprios templates.
Espero que consigam utilizar com facilidade esse notebook!

##### Que Nimb guie seus dados!

In [1]:
pip install regex pyperclip typing

Note: you may need to restart the kernel to use updated packages.


In [2]:
import base

import pyperclip as pyclip

from functools import partial
from typing import Callable, Tuple

In [3]:
def pega_atributos (tipo_acao: str) -> list:
    if tipo_acao == "ataque":
        return [ "nome{}".format(tipo_acao), "descricao{}".format(tipo_acao) ]
    else:
        return [ "name{}".format(tipo_acao), "{}description".format(tipo_acao) ]
    
def pega_funcao (tipo_funcao: int, tipo_acao: str, circulo: str, critico: base.CriticalTypes = None) -> Callable:
    if tipo_acao == "ability":
        return base.rpt_ability if tipo_funcao == 1 else base.create_ability_template
    elif tipo_acao == "power":
        return base.rpt_power if tipo_funcao == 1 else base.create_power_template
    elif tipo_acao == "spell":
        return (
            partial(base.rpt_spell, spell_circle=circulo)
            if tipo_funcao == 1
            else partial(base.create_spell_template, spell_circle=circulo)
        )
    elif tipo_acao == "attack":
        if critico is None:
            return base.rpt_power if tipo_funcao == 1 else base.create_attack_template
        else:
            return (
                partial(base.rpt_attack, critical=critico)
                if tipo_funcao == 1
                else partial(base.create_attack_template, critical=critico)
            )
    
    return None
        
def repete_acao (tipo_acao: str, circulo: str) -> Tuple[Callable, list]:
    atributos = pega_atributos(tipo_acao)
    funcao = pega_funcao(1, tipo_acao, circulo)
    
    return funcao, atributos

def macroniza_acao (tipo_acao: str, circulo: str, critico: base.CriticalTypes) -> Callable:
    funcao = pega_funcao(0, tipo_acao, circulo, critico)        
    return funcao

In [4]:
def cria_descricao (tipo_acao: str, id_acao: str, copia: str = "nao", circulo: str = "1") -> str:
    descreve, atributos = repete_acao(tipo_acao, circulo)
    if descreve is None:
        return "TIPO INVÁLIDO DE AÇÃO"
    
    descricao = base.describe(*descreve(id_acao, atributos))
    if copia == "sim":
        pyclip.copy(descricao)
    
    return descricao

In [5]:
def cria_macro (tipo_acao: str, id_acao: str, copia: str = "nao", circulo: str = "1", critico: str = "DANO") -> str:
    funcao = macroniza_acao(
        tipo_acao, str(circulo), base.CriticalTypes.DAMAGE if critico == "DANO" else base.CriticalTypes.DICE
    )
    
    macro = funcao(id_acao)
    if copia == "sim":
        pyclip.copy(macro)
        
    return funcao(id_acao)

## Início dos Exemplos

In [6]:
cria_descricao("ability", "-MWol3wUlwt1di-xQWnZ", copia="nao")

'**@{repeating_abilities_-MWol3wUlwt1di-xQWnZ_nameability}**: @{repeating_abilities_-MWol3wUlwt1di-xQWnZ_abilitydescription}'

In [7]:
cria_descricao("power", "-MWokal-6PQ7hFQQITAE", copia="nao")

'**@{repeating_powers_-MWokal-6PQ7hFQQITAE_namepower}**: @{repeating_powers_-MWokal-6PQ7hFQQITAE_powerdescription}'

In [8]:
cria_macro("attack", "-MX-d2s12Nz2i6J6qbQY", copia="nao") # Multiplicando o dano dos dados

'&{template:t20-attack}{{character=@{character_name}}}{{attackname=@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_nomeataque}}}{{attackroll=[[1d20cs>@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_margemcriticoataque}+[[@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_ataquepericia}]]+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_bonusataque}]]}} {{damageroll=[[@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danoataque}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_modatributodano}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danoextraataque}]]}} {{criticaldamageroll=[[@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danoataque}*@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_multiplicadorcriticoataque}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_modatributodano}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danoextraataque}]]}}{{typeofdamage=@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_ataquetipodedano}}}{{description=@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_ataquedescricao}}}'

In [9]:
cria_macro("attack", "-MX-d2s12Nz2i6J6qbQY", copia="nao", critico="DADO") # Multiplicando o número de dados

'&{template:t20-attack}{{character=@{character_name}}}{{attackname=@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_nomeataque}}}{{attackroll=[[1d20cs>@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_margemcriticoataque}+[[@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_ataquepericia}]]+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_bonusataque}]]}} {{damageroll=[[@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danoataque}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_modatributodano}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danoextraataque}]]}} {{criticaldamageroll=[[@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danocriticoataque}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_modatributodano}+@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_danoextraataque}]]}}{{typeofdamage=@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_ataquetipodedano}}}{{description=@{repeating_attacks_-MX-d2s12Nz2i6J6qbQY_ataquedescricao}}}'

In [10]:
cria_macro("spell", "-MWojXgYrdLO17N5ljE8", copia="nao", circulo="1")

'&{template:spell}{{character=@{character_name}}}{{spellname=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_namespell}}}{{type=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spelltipo}}}{{execution=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spellexecucao}}}{{duration=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spellduracao}}}{{range=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spellalcance}}}{{targetarea=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spellalvoarea}}}{{resistance=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spellresistencia}}}{{description=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spelldescription}}}{{cd=@{repeating_spells1_-MWojXgYrdLO17N5ljE8_spellcd}}}'

In [11]:
cria_macro("spell", "-MX-h-dXicmUNW88uEC-", copia="nao", circulo="2")

'&{template:spell}{{character=@{character_name}}}{{spellname=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_namespell}}}{{type=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spelltipo}}}{{execution=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spellexecucao}}}{{duration=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spellduracao}}}{{range=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spellalcance}}}{{targetarea=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spellalvoarea}}}{{resistance=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spellresistencia}}}{{description=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spelldescription}}}{{cd=@{repeating_spells2_-MX-h-dXicmUNW88uEC-_spellcd}}}'

In [12]:
# Exemplo com copia, rodo por padrão para não sobrescrever o CTRL+C do usuário
# cria_macro("spell", "-MX-h-dXicmUNW88uEC-", copia="sim", circulo="2")

#### Agora seguem os templates

Remova os # do início da função que deseja e realize as alterações indicadas em ***Entendendo os Templates****

In [13]:
# cria_descricao("{tipo_template}", "{tipo_acao}", "{id_acao}", "{copia}")
# cria_descricao("spell", "{tipo_acao}", "{id_acao}", "{copia}", circulo="{circulo_magia}")

In [14]:
# cria_macro("{tipo_template}", "{id_acao}", "{copia}")
# cria_macro("spell", "{id_acao}", "{copia}", circulo="{circulo_magia}")